In [1]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_excel("Demographic.xlsx")
df.head()


,Region,PC,Description,km^2,Households,Population,Category,Affluent,Connected,Mobile Lifestyle,City Slickers
0,Anglia,AL,St Albans,296,97525,229502,Suburban,0.16,0.13,0.16,0.13
1,Anglia,CB,Cambridge,1818,148838,349949,Suburban,0.22,0.13,0.22,0.13
2,Anglia,CM,Chelmsford,1800,254271,613034,Rural,0.22,0.13,0.22,0.13
3,Anglia,CO,Colchester,1334,164746,377322,Rural,0.18,0.07,0.18,0.07
4,Anglia,EN,Enfield,219,136556,330709,Urban,0.11,0.15,0.11,0.15


In [2]:
# Split into separate dataframes with .copy() to avoid warnings
df_urban = df[df['Category'].str.lower() == 'urban'].copy()
df_rural = df[df['Category'].str.lower() == 'rural'].copy()
df_suburban = df[df['Category'].str.lower() == 'suburban'].copy()


In [3]:
# Affluent & Connected → multiply by Households
# Mobile Lifestyle & City Slickers → multiply by Population

for d in [df_urban, df_rural, df_suburban]:
    d['Affluent'] = pd.to_numeric(d['Affluent'], errors='coerce') * d['Households']
    d['Connected'] = pd.to_numeric(d['Connected'], errors='coerce') * d['Households']
    d['Mobile Lifestyle'] = pd.to_numeric(d['Mobile Lifestyle'], errors='coerce') * d['Population']
    d['City Slickers'] = pd.to_numeric(d['City Slickers'], errors='coerce') * d['Population']


In [4]:
# Tower coverage areas (km^2)
coverage = {
    "urban": np.pi * (1.5**2),      # ~7.07 km^2
    "suburban": np.pi * (4**2),     # ~50.27 km^2
    "rural": np.pi * (9**2)         # ~254.47 km^2
}

df_urban['Towers Needed'] = np.ceil(df_urban['km^2'] / coverage['urban'])
df_rural['Towers Needed'] = np.ceil(df_rural['km^2'] / coverage['rural'])
df_suburban['Towers Needed'] = np.ceil(df_suburban['km^2'] / coverage['suburban'])


In [5]:
# Plan uptake dictionary {plan: {segment: [year1, year2, year3]}}
plans = {
    "LightSpeed Unlimited": {
        "Mobile Lifestyle": [0.005, 0.015, 0.03],
        "City Slickers": [0.005, 0.01, 0.015],
    },
    "LightSpeed Unplugged": {
        "Mobile Lifestyle": [0.01, 0.02, 0.03],
        "City Slickers": [0.015, 0.03, 0.05],
    },
    "LightSpeed Unwired": {
        "Affluent": [0.005, 0.01, 0.03],
        "Connected": [0.005, 0.01, 0.03],
    },
    "Total LightSpeed": {
        "Affluent": [0.0, 0.005, 0.01],
        "Connected": [0.0, 0.005, 0.005],
    }
}


In [6]:
def compute_abs_subs(df_cat):
    abs_counts = {plan: [] for plan in plans.keys()}
    for year in range(3):
        for plan, segments in plans.items():
            total_users = 0
            for seg, pct in segments.items():
                total_users += df_cat[seg].sum() * pct[year]
            abs_counts[plan].append(int(total_users))
    return pd.DataFrame(abs_counts, index=["Year 1", "Year 2", "Year 3"])

abs_urban = compute_abs_subs(df_urban)
abs_rural = compute_abs_subs(df_rural)
abs_suburban = compute_abs_subs(df_suburban)

print("Urban absolute subscribers:\n", abs_urban.head())
print("\nRural absolute subscribers:\n", abs_rural.head())
print("\nSuburban absolute subscribers:\n", abs_suburban.head())


Urban absolute subscribers:
         LightSpeed Unlimited  LightSpeed Unplugged  LightSpeed Unwired  \
Year 1                 15926                 43673                6935   
Year 2                 35958                 87347               13871   
Year 3                 60095                142842               41613   

        Total LightSpeed  
Year 1                 0  
Year 2              6935  
Year 3              8669  

Rural absolute subscribers:
         LightSpeed Unlimited  LightSpeed Unplugged  LightSpeed Unwired  \
Year 1                 20729                 49431                8768   
Year 2                 54216                 98863               17536   
Year 3                100459                156268               52610   

        Total LightSpeed  
Year 1                 0  
Year 2              8768  
Year 3             14164  

Suburban absolute subscribers:
         LightSpeed Unlimited  LightSpeed Unplugged  LightSpeed Unwired  \
Year 1                 2

In [7]:
# Monthly plan prices
plan_prices = {
    "LightSpeed Unlimited": 75,
    "LightSpeed Unplugged": 30,
    "LightSpeed Unwired": 40,
    "Total LightSpeed": 120
}

# Contract duration in months
duration_months = 18


In [8]:
def revenue_by_region(abs_df, region_name):
    results = {}
    for plan, price in plan_prices.items():
        subs = abs_df[plan].sum()   # total subs across 3 years
        revenue = subs * price * duration_months
        results[plan] = revenue
    return pd.DataFrame(results, index=[region_name])

urban_rev = revenue_by_region(abs_urban, "Urban")
rural_rev = revenue_by_region(abs_rural, "Rural")
suburban_rev = revenue_by_region(abs_suburban, "Suburban")


In [9]:
plan_revenue_breakdown = pd.concat([urban_rev, rural_rev, suburban_rev])
plan_revenue_breakdown.loc["Total"] = plan_revenue_breakdown.sum()
plan_revenue_breakdown


,LightSpeed Unlimited,LightSpeed Unplugged,LightSpeed Unwired,Total LightSpeed
Urban,151171650,147885480,44941680,33704640
Rural,236795400,164463480,56818080,49533120
Suburban,267070500,201926520,67638960,56931120
Total,655037550,514275480,169398720,140168880


In [10]:
# Device cost tiers (users thresholds, € per device)
device_cost_tiers = {
    "LightSpeed Unlimited": [(1_000_000, 350), (5_000_000, 330), (10_000_000, 320), (float("inf"), 300)],
    "LightSpeed Unplugged": [(1_000_000, 50), (5_000_000, 48), (10_000_000, 46), (float("inf"), 43)],
    "LightSpeed Unwired":   [(1_000_000, 200), (3_000_000, 185), (10_000_000, 180), (float("inf"), 170)],
    "Total LightSpeed":     [(1_000_000, 350), (5_000_000, 330), (10_000_000, 320), (float("inf"), 300)], # bundle aligned with Unlimited
}


In [11]:
# Subsidy percentages per year
subsidy_schedule = [0.5, 0.75, 1.0]  # Year 1 = 50%, Year 2 = 75%, Year 3 = 100%


In [12]:
def get_device_cost(plan, total_users):
    for threshold, cost in device_cost_tiers[plan]:
        if total_users <= threshold:
            return cost
    return device_cost_tiers[plan][-1][1]


In [13]:
def subsidy_by_region(abs_df, region_name):
    results = {}
    for plan in plans.keys():
        subs_by_year = abs_df[plan].values
        total_users = subs_by_year.sum()
        device_cost = get_device_cost(plan, total_users)

        # Year-wise subsidy
        subsidy = sum(subs_by_year[year] * device_cost * subsidy_schedule[year] for year in range(3))
        results[plan] = subsidy
    return pd.DataFrame(results, index=[region_name])

urban_sub = subsidy_by_region(abs_urban, "Urban")
rural_sub = subsidy_by_region(abs_rural, "Rural")
suburban_sub = subsidy_by_region(abs_suburban, "Suburban")


In [14]:
def tower_costs(df_cat, region_name):
    num_towers = df_cat["Towers Needed"].sum()
    capex = num_towers * 75000
    opex = num_towers * 3000 * 3  # 3 years
    return pd.DataFrame({"Tower Cost": [capex + opex]}, index=[region_name])

urban_tower = tower_costs(df_urban, "Urban")
rural_tower = tower_costs(df_rural, "Rural")
suburban_tower = tower_costs(df_suburban, "Suburban")


In [17]:
# Merge with explicit suffixes to avoid duplicate labels
urban_fin = pd.concat([urban_rev.add_suffix(" Revenue"), urban_sub.add_suffix(" Subsidy")], axis=1)
rural_fin = pd.concat([rural_rev.add_suffix(" Revenue"), rural_sub.add_suffix(" Subsidy")], axis=1)
suburban_fin = pd.concat([suburban_rev.add_suffix(" Revenue"), suburban_sub.add_suffix(" Subsidy")], axis=1)


# Add tower costs evenly across plans (split by plan share of revenue)
def allocate_tower_cost(fin_df, tower_df):
    # total revenue = sum of all plan revenues
    revenue_cols = [c for c in fin_df.columns if "Revenue" in c]
    total_rev = fin_df[revenue_cols].iloc[0].sum()
    tower_total = tower_df["Tower Cost"].iloc[0]

    alloc = {}
    for col in revenue_cols:
        plan = col.replace(" Revenue", "")
        rev = fin_df[col].iloc[0]
        alloc[plan] = tower_total * (rev / total_rev)
    return pd.DataFrame(alloc, index=["Tower Cost"])

urban_tower_alloc = allocate_tower_cost(urban_fin, urban_tower)
rural_tower_alloc = allocate_tower_cost(rural_fin, rural_tower)
suburban_tower_alloc = allocate_tower_cost(suburban_fin, suburban_tower)


# Final profit = Revenue – Subsidy – Allocated Tower Cost
def compute_profit(fin_df, tower_alloc, region):
    profit = {}
    for plan in plan_prices.keys():
        revenue = fin_df[f"{plan} Revenue"].iloc[0]
        subsidy = fin_df[f"{plan} Subsidy"].iloc[0]
        tower = tower_alloc[plan].iloc[0]
        profit[plan] = revenue - subsidy - tower
    return pd.DataFrame(profit, index=[region])

urban_profit = compute_profit(urban_fin, urban_tower_alloc, "Urban")
rural_profit = compute_profit(rural_fin, rural_tower_alloc, "Rural")
suburban_profit = compute_profit(suburban_fin, suburban_tower_alloc, "Suburban")



In [18]:
profit_breakdown = pd.concat([urban_profit, rural_profit, suburban_profit])
profit_breakdown.loc["Total"] = profit_breakdown.sum()
profit_breakdown


,LightSpeed Unlimited,LightSpeed Unplugged,LightSpeed Unwired,Total LightSpeed
Urban,8.597330e+07,1.051313e+08,2.434979e+07,2.172904e+07
Rural,1.579132e+08,1.337446e+08,3.658334e+07,3.686423e+07
Suburban,1.605181e+08,1.507293e+08,3.903986e+07,3.860497e+07
Total,4.044047e+08,3.896051e+08,9.997300e+07,9.719824e+07
